# Сериализация и десериализация данных в C++

Сериализация данных - процесс преобразования данных, хранящихся в оперативной памяти, в текстовый или бинарный формат.

Десериализация (структурирование) данных - обратный процесс: преобразование данных, хранящихся в текстовом или бинарном формате, в структуру в оперативной памяти.

Бинарный формат сериализации подразумевает под собой преобразование данных в последовательность байтов. 
Как правило, данный формат сериализации будет занимать меньший объём памяти по сравнении с текстовым, 
но при этом будет абсолютно нечитаемым для человека.

Текстовый формат сериализации подразумает под собой преобразование данных в текстовые данные, которые удобно воспринимать человеку.

Например переменная, 
```cpp
char x = 10;
```

в бинарном формате будет записана как байт 0xA, а в текстовом, как 2 символа: '1' и '0' - "10".

Соответственно, в бинарном формате данная переменная займёт 1 байт в памяти, а в текстовом - 2 байта.

## Бинарная сериализация данных

Для начала научимся сериализовать переменную в бинарный формат и сохранять её в файл.

In [1]:
#include <iostream>
// для записи в файл включим fstream
#include <fstream>

using namespace std;

In [2]:
char x = 123;

Открываем файл для записи

In [3]:
ofstream out("file.bin");

Для записи последовательности байтов в файл используется метод
```cpp
ofstream.write(const char* data, size_t size);
```
где data - данные, а size - размер данных в байтах.

Записываем данные в файл.

In [4]:
out.write(&x, sizeof(x));
// оператор sizeof(T) позволяет получить размер T в байтах

@0x7f41583fc030

Обязательно закрываем файл и сохраняем его

In [5]:
out.close();

Теперь в файле "file.bin" записано значение переменной **x** в байтах.

Теперь структурируем эту переменную из файла.

Открываем файл для чтения

In [6]:
ifstream in("file.bin");

Для чтения последовательности байтов из файла исползуется метод
```cpp
ifstream.read(char* to, size_t size);
```
где to - переменная для чтения, а size - количество читаемых байтов.

Читаем данные из файла

In [7]:
char value;
in.read(&value, sizeof(char));

Также закрываем файл и сохраняем его

In [8]:
in.close();

Проверим, совпадает ли значение **value** со значением **x**.

In [9]:
cout << x << ' ' << value << ' ' << (x == value) << endl;

{ { 1


Данные сопадают, соответственно, сериализация данных прошла успешно.

### Сериализация других типов данных

Что делать, если нужно сериализовать данные типа, отличного от char?

Алгоритм действий тот же, но, т. к. методы write и read принимают указатель на тип char, то необходимо преобразовать сериализуемый тип в char.

Это можно сделать с помощью оператора reinterpret_cast.

Оператор reinterpret_cast\<NewType>(data) сообщает компилятору о том, data, теперь нужно воспринимать как тип данных NewType.

Аналогом этого оператора является преобразование типов из языка *C*: __(NewType) data__.

**Данный оператор никак не преобразует данные!
После его применения последовательность байтов, на которые ссылается data, будет обрабатываться, как последовательность байтов типа NewType!**

In [10]:
int value = 10;

In [11]:
ofstream out("file2.bin");
out.write(reinterpret_cast<char*>(&value), sizeof(value));
out.close();

Теперь считаем данные из файла "file2.bin"

In [12]:
int x;
ifstream in("file2.bin");
in.read((char*)&x, sizeof(x)); // (char*)&x аналогично reinterpret_cast<char*>(&x)
in.close();

@0x7f41583fc658

In [13]:
cout << x << ' ' << value << ' ' << (x == value) << endl;

10 10 1


Всё совпадает

### Способы открытия файла

При инициализации объекта ifstream или ofstream в качестве второго аргумента можно выбрать способ открытия этого файла.

```cpp
fstream f(const char* file, ios_base::openmode open_mode);
```

#### ios::app

При таком режиме открытия данные всегда будут записываться в конец файла.

#### ios::ate

При открытии файла указатель на запись ставится в конец файла, но его можно передвинуть с помощью метода seek.

#### ios::binary

При таком режиме открытия все байты будут записаны без изменений.

*Пояснение:*

На Linux переход на новую строку обозначается символом '\n'.
На Windows - двумя символами "\r\n".

Если открыть без указания этого способа, то на Windows при записи байта, отвечающего за '\n', перед ним автоматически добавится байт, отвечающий за символ '\r'.

Если открыть этот файл для чтения на Linux, то данные могут считаться неверно, т. к. перед каждым символом '\n' будет добавлен символ '\r'.

#### ios::in

Открытие файла для чтения из него данных.

#### ios::out

Открытие файла для записи в него данных.

#### ios::trunc

Очищает файл при открытии.

Сочетать способы открытия можно с помощью оператора **|**, например **ios::out | ios::trunc | ios::binary** - открытие файла на запись с предварительной очисткой данных в нём + бинарный режим.

### Сериализация объектов

Как правило сериализуют не отдельные переменные, а целые структуры данных.

Попробуем сериализовать небольшой контейнер, основанный на std::vector.

In [14]:
template<class T>
class List {
    vector<T> _data;

public:
    List() = default;

    List(size_t size) : _data(size) {}

    List(const char *file) {
        // конструктор из файла (десериализация)
        ifstream f(file, ios::binary);
        size_t size;
        f.read((char *) &size, sizeof(size_t));
        for (size_t i = 0; i < size; i++) {
            T tmp;
            f.read((char *) &tmp, sizeof(T));
            _data.push_back(tmp);
        }
        f.close();
    }

    void append(T &data) {
        _data.push_back(data);
    }

    void append(T &&data) {
        _data.push_back(data);
    }

    T pop() {
        return _data.pop_back();
    }
    
    void print() {
    for (T &item : _data) cout << item << ' ';
        cout << endl;
    }

    void serialize(const char *file) {
        // метод сериализации структуры (сохранения её в файл)
        ofstream out(file, ios::binary | ios::trunc);
        serialize(out);
        out.close();
    }

    void serialize(ostream &out) {
        // метод сериализации структуры (сохранение в выходной буфер)
        size_t size = _data.size();
        out.write((char *) &size, sizeof(size_t)); // сохранение длины списка
        for (T &item: _data) // сохранение данных
            out.write((char *) &item, sizeof(T));
    }

    static List<T> deserialize(const char *file) {
        // метод десериализации данных (получение структуры данных из файла)
        ifstream in(file, ios::binary);
        auto object = deserialize(in);
        in.close();
        return object;
    }

    static List<T> deserialize(istream &in) {
        // метод десериализации данных (получение структуры данных из входного буфера)
        List<T> object;
        size_t size;
        in.read((char *) &size, sizeof(size));
        for (size_t i = 0; i < size; i++) {
            T tmp;
            in.read((char *) &tmp, sizeof(T));
            object.append(tmp);
        }
        return object;
    }

    T &operator[](size_t index) {
        return _data[index];
    }
};

Инициализируем и "заполним" структуру

In [15]:
List<int> my_list;
for (int i = 0; i < 10; i++) 
    my_list.append(i);

Просмотрим содержимое структуры

In [16]:
my_list.print();

0 1 2 3 4 5 6 7 8 9 


Сериализуем структуру в файл "list.bin"

In [17]:
my_list.serialize("list.bin");

Попробуем загрузить структуру из файла несколькими способами

1) Через конструктор

In [18]:
List<int> my_list2("list.bin");
my_list2.print();

0 1 2 3 4 5 6 7 8 9 


Сработало

2) Через статичный метод deserialize

In [19]:
auto my_list3 = List<int>::deserialize("list.bin");
my_list3.print();

0 1 2 3 4 5 6 7 8 9 


Тоже сработало, данные не исказились

## Текстовая сериализация

Для того, чтобы сериализовать данные в файл в текстовом виде, нужно использовать оператор **<<**, чтобы десериализовать данные из файла - оператор **>>**.

In [20]:
ofstream out("text.txt");

In [21]:
int x = 10;

In [22]:
out << x << endl;

In [23]:
out.close();

В файле записана строка "10".

Считаем данные из файла.

In [24]:
ifstream in("text.txt");

In [25]:
int value;
in >> value;

In [26]:
in.close();

In [27]:
cout << value << endl;

10
